In [3]:
require(ggplot2)
require(repr)
require(dplyr)

Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [4]:
options(repr.plot.width=6 ,repr.plot.height=4)

In [41]:
# 클래스 정의
setClass("Index",
    slots=c(
        indexID="character",
        startDate="Date",
        tabulate="logical",
        annualizedRate="numeric",
        yearlyIndex="vector",
        monthlyIndex="vector",
        seasonality="vector"
    ),
    prototype=list(
        indexID="XXXXXX",
        startDate=as.Date("2012-01-01"),
        tabulate=FALSE,
        annulaizedRate=0.02,
        yearlyIndex=vector(),
        monthlyIndex=vector(),
        seasonality=rep(1,12)
    )
)

In [43]:
# ID 설정하기
setGeneric("setID<-", function(this, value, ...) standardGeneric("setID<-"))
setReplaceMethod("setID", signature("Index", "character"), function(this, value){
  this@indexID <- as.character(value)  
  return(this)
})       

[1] "setID<-"

In [44]:
# 시작일 설정하기
setGeneric("setStartDate<-", function(this, value) standardGeneric("setStartDate<-"))
setReplaceMethod("setStartDate", signature("Index", "Date"), function(this, value){
    this@startDate <- as.Date(value)
    return(this)
})

[1] "setStartDate<-"

In [45]:
# 테이블 입력 여부 설정하기
setGeneric("setTabulate<-", function(this, value, ...) standardGeneric("setTabulate<-"))
setReplaceMethod("setTabulate", signature("Index", "logical"), function(this, value){
    this@tabulate <- value
    return(this)
})

[1] "setTabulate<-"

In [46]:
# 연단위 일정한 인덱스 설정하기
setGeneric("setAnnualizedRate<-", function(this, value, ...) standardGeneric("setAnnualizedRate<-"))
setReplaceMethod("setAnnualizedRate", signature("Index", "numeric"), function(this, value){
    this@annualizedRate <- value
    return(this)
})

[1] "setAnnualizedRate<-"

In [47]:
# 연단위 인덱스 설정하기
setGeneric("setYearlyIndex<-", function(this, value, ...) standardGeneric("setYearlyIndex<-"))
setReplaceMethod("setYearlyIndex", signature("Index", "vector"), function(this, value){
    this@yearlyIndex <- value
    this@tabulate <- TRUE
    return(this)
})

[1] "setYearlyIndex<-"

In [50]:
# 월단위 인덱스 설정하기
setGeneric("setMonthlyIndex<-", function(this, value, ...) standardGeneric("setMonthlyIndex<-"))
setReplaceMethod("setMonthlyIndex", signature("Index", "vector"), function(this, value){
    this@monthlyIndex <- value
    this@tabulate <- TRUE
    return(this)
})

[1] "setMonthlyIndex<-"

In [179]:
# 계절성 지수 설정하기
setGeneric("setSeasonality<-", function(this, value, ...) standardGeneric("setSeasonality<-"))
setReplaceMethod("setSeasonality", signature("Index", "vector"), function(this, value){
    this@seasonality <- value
    return(this)
})        

[1] "setSeasonality<-"

In [180]:
# 인덱스 설정하기
setGeneric("setIndex", function(object, ...) standardGeneric("setIndex"))
setMethod("setIndex", signature("Index"), function(object){
    tryCatch({
        yearlylen <- length(object@yearlyIndex)
    
        #1. Index를 annualized Index로 만드는 경우
        if(object@tabulate==FALSE){
            object@monthlyIndex <- cumprod(c(1, rep((1+object@annualizedRate)^(1/12), 359)))

            # Seasonality Index 반영
            for(i in 1:length(object@monthlyIndex)){
                mth <- (i-1)%%12 + 1
                object@monthlyIndex[i] <- object@monthlyIndex[i]*object@seasonality[mth]
            }

        #2. Monthly Index가 없고 Yearly Index는 존재하는 경우
        }else if(length(object@monthlyIndex)==0 && yearlylen>0){

            # yearly Index가 30개 이하면 30개 맞춰줌(최소 30년)
            if(yearlylen < 30){
            warning(paste0("Index ", object@indexID, ": yearly index input is less than 30 years and is extrapolated using annulaized rate."))
            object@yearlyIndex <- round(c(object@yearlyIndex, cumprod(c(object@yearlyIndex[yearlylen]*(1+object@annualizedRate), rep(1+object@annualizedRate, 30-yearlylen-1)))),2)
            }

            # Yearly Index 중에 음수인 애가 있을 때 ERROR
            if(any(object@yearlyIndex < 0)) stop("yearlyIndex cannot be negative.")

            # Yearly Index를 가지고 Monthly Index 생성
            # 30년 이하면 이상하게 나오는 듯?
            object@monthlyIndex <- rep(1,360)
            object@monthlyIndex[1] <- object@yearlyIndex[1]
            for(i in 13:length(object@monthlyIndex)){
                yr <- ceiling(i/12)
                rte <- (object@yearlyIndex[yr]/object@yearlyIndex[yr-1])^(1/12)
                object@monthlyIndex[i] <- object@monthlyIndex[i-1]*rte
            }

            # Seasonality Index 반영
            for(i in 1:length(object@monthlyIndex)){
                mth <- (i-1)%%12 + 1
                object@monthlyIndex[i] <- object@monthlyIndex[i]*object@seasonality[mth]
            }

        #3. Yearly Index도 존재하지 않는 경우
        }else if(length(object@monthlyIndex)==0){
            warning(paste0("Index ", object@indexID, ": No index value provided. Index value will be set to 1 with seasonal adjustment."))

            object@monthlyIndex <- rep(1,360)
            # Seasonality Index 반영
            for(i in 1:length(object@monthlyIndex)){
                mth <- (i-1)%%12 + 1
                object@monthlyIndex[i] <- object@monthlyIndex[i]*object@seasonality[mth]
            }
        }

        # Monthly Index 중에 음수인 애가 있을 때 ERROR
        if(any(object@yearlyIndex < 0)) stop("monthlyIndex cannot be negative.")

        gc()
        return(object)
    }, error=function(err){
        print(paste0(">>>Critical Error for ", "Index ", object@indexID, ": ", err))
        gc()
        return(-1)
    })
})

[1] "setIndex"

In [264]:
# 인덱스 이동
setGeneric("shiftIndex", function(object, ...) standardGeneric("shiftIndex"))
setMethod("shiftIndex", signature("Index"), function(object, newStartDate, newEndDate){
    tryCatch({
        newyear <- as.numeric(substr(as.character(newStartDate), 1, 4))
        newmonth <- as.numeric(substr(as.character(newStartDate), 6, 7))
        endyear <- as.numeric(substr(as.character(newEndDate), 1, 4))
        endmonth <- as.numeric(substr(as.character(newEndDate), 6, 7))
        startyear <- as.numeric(substr(as.character(object@startDate), 1, 4))
        startmonth <- as.numeric(substr(as.character(object@startDate), 6, 7))
        sindex <- (newyear-startyear)*12+(newmonth-startmonth)+1
        nlen <- (endyear-newyear)*12+(endmonth-newmonth)+1

        newMI <- c(rep(0,max(0,1-sindex)), object@monthlyIndex[max(1,sindex):min(length(object@monthlyIndex),length(object@monthlyIndex)+sindex-1)])
        #newMI <- c(rep(0,max(0,1-sindex)),object@monthlyIndex[max(1,sindex):min(length(object@monthlyIndex),length(object@monthlyIndex)+sindex-1)])
        if(nlen>length(newMI)){
            newMI <- c(newMI, rep(0,nlen-length(newMI)))
        }else{
            newMI <- newMI[1:nlen]
        }
        
        gc()
        return(newMI)
    }, error=function(err){
        print(paste0(">>>Critical Error for shifting index values", "Index ", object@indexID, ": ", err))
        gc()
        return(-1)
    })
})

[1] "shiftIndex"

In [272]:
# 인덱스의 문자화(?)
setGeneric("toString", function(object, ...) standardGeneric("toString"))
setMethod("toString", signature("Index"), function(object) return(paste0("Index ", object@indexID, " start date=", object@startDate)))

[1] "toString"

In [275]:
# 인스턴스 생성
idx <- new("Index")

setID(idx) <- 'RX-78'
setStartDate(idx) <- as.Date('2017-11-30')
setAnnualizedRate(idx) <- 0.045

idx <- setIndex(idx)
idx

An object of class "Index"
Slot "indexID":
[1] "RX-78"

Slot "startDate":
[1] "2017-11-30"

Slot "tabulate":
[1] FALSE

Slot "annualizedRate":
[1] 0.045

Slot "yearlyIndex":
logical(0)

Slot "monthlyIndex":
  [1] 1.000000 1.003675 1.007363 1.011065 1.014780 1.018510 1.022252 1.026009
  [9] 1.029779 1.033564 1.037362 1.041174 1.045000 1.048840 1.052694 1.056563
 [17] 1.060446 1.064343 1.068254 1.072179 1.076119 1.080074 1.084043 1.088027
 [25] 1.092025 1.096038 1.100066 1.104108 1.108166 1.112238 1.116325 1.120427
 [33] 1.124545 1.128677 1.132825 1.136988 1.141166 1.145360 1.149569 1.153793
 [41] 1.158033 1.162289 1.166560 1.170847 1.175149 1.179468 1.183802 1.188152
 [49] 1.192519 1.196901 1.201299 1.205714 1.210145 1.214592 1.219055 1.223535
 [57] 1.228031 1.232544 1.237073 1.241619 1.246182 1.250761 1.255358 1.259971
 [65] 1.264601 1.269248 1.273912 1.278594 1.283292 1.288008 1.292742 1.297492
 [73] 1.302260 1.307046 1.311849 1.316670 1.321508 1.326364 1.331239 1.336131
 [81] 1.34104